In [1]:
from __future__ import print_function
import six
import sys
import os
from optparse import OptionParser
from keras.models import load_model, Model, model_from_json
from argparse import ArgumentParser
from keras import backend as K
from keras.layers import Input
import numpy as np
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import pandas as pd
from keras.optimizers import Adam, Nadam
from callbacks import all_callbacks
import pandas as pd
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import yaml
from train import parse_config, get_features

Using TensorFlow backend.


In [2]:
from collections import namedtuple
Option = namedtuple("MyStruct", "inputFile tree config outputDir")

options = Option(
    inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_conv1d.yml',
    outputDir = '../conv1d_weights/'
)

print("Loading configuration from", options.config)
config = open(options.config, 'r')
yamlConfig =  yaml.load(config, Loader=yaml.FullLoader)
print(yamlConfig)

Loading configuration from train_config_conv1d.yml
{'Inputs': ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j1_deltaR', 'j1_costhetarel', 'j1_pdgid', 'j_index'], 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index'], 'KerasModel': 'conv1d_model', 'KerasModelRetrain': 'conv1d_model_constraint', 'KerasLoss': 'categorical_crossentropy', 'L1Reg': 0.0001, 'NormalizeInputs': 1, 'InputType': 'Conv1D', 'MaxParticles': 100}


In [3]:
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Keys: <KeysViewHDF5 ['t_allpar_new']>
load 24816
load 0
load 1000
load 2000
load 3000
load 4000
load 5000
load 6000
load 7000
load 8000
load 9000
load 10000
load 11000
load 12000
load 13000
load 14000
load 15000
load 16000
load 17000
load 18000
load 19000
load 20000
load 21000
load 22000
load 23000
load 24000


In [4]:
from models.models import conv1d_model
keras_model = conv1d_model(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'] )

startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])


callbacks=all_callbacks(stop_patience=1000, 
                        lr_factor=0.5,
                        lr_patience=10,
                        lr_epsilon=0.000001, 
                        lr_cooldown=2, 
                        lr_minimum=0.0000001,
                        outputDir=options.outputDir)

keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:251: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=32, kernel_size=40, strides=1, padding="same", kernel_initializer="he_normal", use_bias=True, name="conv1_relu", activation="relu", kernel_regularizer=<keras.reg...)`
  activation = 'relu', W_regularizer=l1(l1Reg))(Inputs)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:254: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=32, kernel_size=40, strides=1, padding="same", kernel_initializer="he_normal", use_bias=True, name="conv2_relu", activation="relu", kernel_regularizer=<keras.reg...)`
  activation = 'relu', W_regularizer=l1(l1Reg))(x)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:257: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=32, kernel_size=40, strides=1, padding="same", kernel_initializer="he_normal", use_bias=True, name="


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 7)            0         
_________________________________________________________________
conv1_relu (Conv1D)          (None, 100, 32)           8992      
_________________________________________________________________
conv2_relu (Conv1D)          (None, 100, 32)           40992     
_________________________________________________________________
conv3_relu (Conv1D)          (None, 100, 32)           40992     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
fc1_relu (Dense)             (None, 50)                160050    
_________________________________________________________________
output_softmax (Dense)       (None, 5)                 255       
Total par

C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:260: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="lecun_uniform", name="fc1_relu", kernel_regularizer=<keras.reg...)`
  name='fc1_relu', W_regularizer=l1(l1Reg))(x)
C:\Users\Owner\JupyterNotebooks\keras-training\train/..\models\models.py:262: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", kernel_initializer="lecun_uniform", name="output_softmax", kernel_regularizer=<keras.reg...)`
  name='output_softmax', W_regularizer=l1(l1Reg))(x)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 14889 samples, validate on 4963 samples







Epoch 1/100
14889/14889 [==============================] - 30s 2ms/step - loss: 1.5295 - acc: 0.6891 - val_loss: 1.0065 - val_acc: 0.8201

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00001: val_loss improved from inf to 1.00650, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 1.00650, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00001: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00001: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***


Epoch 2/100
14889/14889 [==============================] - 30s 2ms/step - loss: 0.8459 - acc: 0.8776 - val_loss: 0.7409 - val_acc: 0.9075

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00002: val_loss impr

14889/14889 [==============================] - 33s 2ms/step - loss: 0.5052 - acc: 0.9991 - val_loss: 0.5056 - val_acc: 0.9986

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00010: val_loss improved from 0.51217 to 0.50560, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00010: val_loss improved from 0.51217 to 0.50560, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00010: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00010: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

Epoch 00010: saving model to ../conv1d_weights//KERAS_check_model_epoch10.h5

***callbacks end***

Epoch 11/100
14889/14889 [==============================] - 34s 2ms/step - loss: 0.4994 - acc: 0.9991 - val_loss: 0.5001 - val_acc: 0.9986

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00011: val_loss improved from 0.50560 to 0.50012, saving model to ../conv1d_weights//KERAS_check_bes

...

14889/14889 [==============================] - 57s 4ms/step - loss: 0.3888 - acc: 0.9998 - val_loss: 0.3902 - val_acc: 0.9994

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00036: val_loss improved from 0.39390 to 0.39022, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00036: val_loss improved from 0.39390 to 0.39022, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00036: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00036: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 37/100
14889/14889 [==============================] - 57s 4ms/step - loss: 0.3850 - acc: 0.9998 - val_loss: 0.3868 - val_acc: 0.9992

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00037: val_loss improved from 0.39022 to 0.38679, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00037: val_loss improved from 0.39022 to 0.38679, saving mod

14889/14889 [==============================] - 57s 4ms/step - loss: 0.3417 - acc: 0.9999 - val_loss: 0.3436 - val_acc: 0.9996

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00049: val_loss improved from 0.34676 to 0.34358, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00049: val_loss improved from 0.34676 to 0.34358, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00049: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00049: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 50/100
14889/14889 [==============================] - 57s 4ms/step - loss: 0.3383 - acc: 0.9999 - val_loss: 0.3406 - val_acc: 0.9992

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00050: val_loss improved from 0.34358 to 0.34058, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00050: val_loss improved from 0.34358 to 0.34058, saving mod

14889/14889 [==============================] - 56s 4ms/step - loss: 0.2993 - acc: 0.9999 - val_loss: 0.3012 - val_acc: 0.9994

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00062: val_loss improved from 0.30422 to 0.30116, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00062: val_loss improved from 0.30422 to 0.30116, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00062: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00062: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 63/100
14889/14889 [==============================] - 56s 4ms/step - loss: 0.2962 - acc: 0.9999 - val_loss: 0.2980 - val_acc: 0.9996

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00063: val_loss improved from 0.30116 to 0.29800, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00063: val_loss improved from 0.30116 to 0.29800, saving mod

Epoch 75/100
14889/14889 [==============================] - 51s 3ms/step - loss: 0.2612 - acc: 0.9999 - val_loss: 0.2629 - val_acc: 0.9996

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00075: val_loss improved from 0.26625 to 0.26295, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00075: val_loss improved from 0.26625 to 0.26295, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00075: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00075: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 76/100
14889/14889 [==============================] - 57s 4ms/step - loss: 0.2585 - acc: 0.9998 - val_loss: 0.2604 - val_acc: 0.9994

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00076: val_loss improved from 0.26295 to 0.26041, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00076: val_loss improved from 0.26295 to 0.2604


Epoch 88/100
14889/14889 [==============================] - 53s 4ms/step - loss: 0.2275 - acc: 0.9998 - val_loss: 0.2295 - val_acc: 0.9990

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00088: val_loss improved from 0.23160 to 0.22955, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00088: val_loss improved from 0.23160 to 0.22955, saving model to ../conv1d_weights//KERAS_check_best_model_weights.h5

Epoch 00088: saving model to ../conv1d_weights//KERAS_check_model_last.h5

Epoch 00088: saving model to ../conv1d_weights//KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 89/100
14889/14889 [==============================] - 48s 3ms/step - loss: 0.2251 - acc: 0.9999 - val_loss: 0.2269 - val_acc: 0.9992

***callbacks***
saving losses to ../conv1d_weights/losses.log

Epoch 00089: val_loss improved from 0.22955 to 0.22687, saving model to ../conv1d_weights//KERAS_check_best_model.h5

Epoch 00089: val_loss improved from 0.22955 to 0.226